<a href="https://colab.research.google.com/github/RebeccaBZ/Hotel_Booking_Case_Study/blob/main/Analysis_booking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
pip install pyspark

In [9]:
from pyspark.sql import SparkSession
spark= SparkSession.builder\
.appName("booking_analysis")\
.getOrCreate()

In [10]:
df= spark.read.csv("hotels_booking.csv", header= True)

In [11]:
df.printSchema()
df.columns

# no changes to the column names is required since they are already written in the correct format

root
 |-- hotel_name: string (nullable = true)
 |-- location: string (nullable = true)
 |-- distance_from_beirut: string (nullable = true)
 |-- number_reviews: string (nullable = true)
 |-- distance_from_beach: string (nullable = true)
 |-- sustainability_level: string (nullable = true)
 |-- room_type: string (nullable = true)
 |-- cancellation_policy: string (nullable = true)
 |-- hotel_policies: string (nullable = true)
 |-- rating: string (nullable = true)
 |-- rating_tile: string (nullable = true)
 |-- price: string (nullable = true)
 |-- additional_charges: string (nullable = true)
 |-- link: string (nullable = true)



['hotel_name',
 'location',
 'distance_from_beirut',
 'number_reviews',
 'distance_from_beach',
 'sustainability_level',
 'room_type',
 'cancellation_policy',
 'hotel_policies',
 'rating',
 'rating_tile',
 'price',
 'additional_charges',
 'link']

In [12]:
rows= df.count()
print("The number of rows in the schema is:", rows)
# counted the number of rows but there is one additional since the number of hotels available is 39 and not 40 in the city of beirut
# this is because an hotel is pinned and hence appears in both pages counting it twice

df= df.dropDuplicates()
rows2= df.count()
print("The edited number of rows in the schema is:", rows2)
# the above dosn't drop out the duplicate rows since the link has minor changes

# I will remove the second one, any would do the job since same data would be displayed by both links
df= df.filter(df["link"]!="https://www.booking.com/hotel/lb/rawsheh-51.en-gb.html?label=gen173nr-1FCAMoggFCBmJlaXJ1dEgJWARoggGIAQGYAQm4ARfIAQzYAQHoAQH4AQKIAgGoAgO4AsibjKgGwAIB0gIkMTcyNDkzZmUtZWExNS00MThjLWI2OGUtODRmMmI2ZmZiNTI22AIF4AIB&aid=304142&ucfs=1&arphpl=1&checkin=2024-08-05&checkout=2024-08-10&dest_id=-801546&dest_type=city&group_adults=2&req_adults=2&no_rooms=1&group_children=0&req_children=0&hpos=1&hapos=26&sr_order=popularity&srpvid=689563ce8ce606a1&srepoch=1694959902&soh=1&from_sustainable_property_sr=1&from=searchresults#no_availability_msg")
rows3= df.count()
print("The edited number of rows in the schema is:", rows3)

df.show()

The number of rows in the schema is: 40
The edited number of rows in the schema is: 40
The edited number of rows in the schema is: 40
+--------------------+--------------------+--------------------+--------------+-------------------+--------------------+--------------------+-------------------+--------------------+------+------------+------------------+--------------------+--------------------+
|          hotel_name|            location|distance_from_beirut|number_reviews|distance_from_beach|sustainability_level|           room_type|cancellation_policy|      hotel_policies|rating| rating_tile|             price|  additional_charges|                link|
+--------------------+--------------------+--------------------+--------------+-------------------+--------------------+--------------------+-------------------+--------------------+------+------------+------------------+--------------------+--------------------+
|Radisson Blu Hote...|     Verdun , Beirut|    2 km from centre|   980 rev

In [13]:
from pyspark.sql import functions as F

df = df.withColumn("number_reviews", F.regexp_extract(df["number_reviews"], r"(\d+)", 1))
df= df.withColumn("number_reviews", F.col("number_reviews").cast("Integer"))

df.printSchema()

df.show()

root
 |-- hotel_name: string (nullable = true)
 |-- location: string (nullable = true)
 |-- distance_from_beirut: string (nullable = true)
 |-- number_reviews: integer (nullable = true)
 |-- distance_from_beach: string (nullable = true)
 |-- sustainability_level: string (nullable = true)
 |-- room_type: string (nullable = true)
 |-- cancellation_policy: string (nullable = true)
 |-- hotel_policies: string (nullable = true)
 |-- rating: string (nullable = true)
 |-- rating_tile: string (nullable = true)
 |-- price: string (nullable = true)
 |-- additional_charges: string (nullable = true)
 |-- link: string (nullable = true)

+--------------------+--------------------+--------------------+--------------+-------------------+--------------------+--------------------+-------------------+--------------------+------+------------+------------------+--------------------+--------------------+
|          hotel_name|            location|distance_from_beirut|number_reviews|distance_from_beach|susta

In [14]:
df= df.withColumn("rating", F.col("rating").cast("Double"))

In [15]:
# usually about 40% of people who booked a hotel room, leave a review hence we predict the number of bookings done up till this date for each hotel
df= df.withColumn("number_bookings", ((100*F.col("number_reviews"))/40))
df.show()



+--------------------+--------------------+--------------------+--------------+-------------------+--------------------+--------------------+-------------------+--------------------+------+------------+------------------+--------------------+--------------------+---------------+
|          hotel_name|            location|distance_from_beirut|number_reviews|distance_from_beach|sustainability_level|           room_type|cancellation_policy|      hotel_policies|rating| rating_tile|             price|  additional_charges|                link|number_bookings|
+--------------------+--------------------+--------------------+--------------+-------------------+--------------------+--------------------+-------------------+--------------------+------+------------+------------------+--------------------+--------------------+---------------+
|Radisson Blu Hote...|     Verdun , Beirut|    2 km from centre|           980|   400 m from beach|Travel Sustainabl...|       Standard Room|                N/A

In [16]:
# extract the nb level
df= df.replace("N/A","0")
df = df.withColumn("sustainability_nb", F.regexp_extract(df["sustainability_level"], r"(\d+)", 1))
df.printSchema()

df.show()

root
 |-- hotel_name: string (nullable = true)
 |-- location: string (nullable = true)
 |-- distance_from_beirut: string (nullable = true)
 |-- number_reviews: integer (nullable = true)
 |-- distance_from_beach: string (nullable = true)
 |-- sustainability_level: string (nullable = true)
 |-- room_type: string (nullable = true)
 |-- cancellation_policy: string (nullable = true)
 |-- hotel_policies: string (nullable = true)
 |-- rating: double (nullable = true)
 |-- rating_tile: string (nullable = true)
 |-- price: string (nullable = true)
 |-- additional_charges: string (nullable = true)
 |-- link: string (nullable = true)
 |-- number_bookings: double (nullable = true)
 |-- sustainability_nb: string (nullable = true)

+--------------------+--------------------+--------------------+--------------+-------------------+--------------------+--------------------+-------------------+--------------------+------+------------+------------------+--------------------+--------------------+---------

In [17]:
df= df.withColumnRenamed("rating_tile", "rating_title")

df = df.withColumn(
    "summary",
     F.when(
        (F.col("sustainability_nb") == "3") &
        ((F.col("rating_title") == "Fabulous") | (F.col("rating_title") == "Superb")),
        "Highly Recommended"
    )
    .when(
        (F.col("sustainability_nb") == "3") | (F.col("rating_title").isin("Fabulous", "Superb")),
        "Moderately Recommended"
    )
    .when(
        (F.col("sustainability_nb") == "2") | (F.col("rating_title") == "Very Good"),
        "Recommended"
    )
    .when(
        (F.col("sustainability_nb") == "1") | (F.col("rating_title") == "Good"),
        "Slightly Recommended"
    )
    .otherwise("Not Recommended")
)

df.show()

+--------------------+--------------------+--------------------+--------------+-------------------+--------------------+--------------------+-------------------+--------------------+------+------------+------------------+--------------------+--------------------+---------------+-----------------+--------------------+
|          hotel_name|            location|distance_from_beirut|number_reviews|distance_from_beach|sustainability_level|           room_type|cancellation_policy|      hotel_policies|rating|rating_title|             price|  additional_charges|                link|number_bookings|sustainability_nb|             summary|
+--------------------+--------------------+--------------------+--------------+-------------------+--------------------+--------------------+-------------------+--------------------+------+------------+------------------+--------------------+--------------------+---------------+-----------------+--------------------+
|Radisson Blu Hote...|     Verdun , Beirut|

In [18]:
df = df.withColumn("additional_charges", F.regexp_replace(F.col("additional_charges"), "[,](?=\\d+)", ""))

df = df.withColumn("charges", F.regexp_extract(df["additional_charges"], r"(\d+)", 1))
df= df.withColumn("charges", F.col("charges").cast("Integer"))
df= df.withColumn("charges_converted", F.col("charges")* 0.0585)
df= df.withColumn("charges_converted", F.round("charges_converted"))
df= df.withColumn("price", F.round("price"))
df= df.drop("sustainability_level", "additional_charges", "charges")
df = df.withColumn("average_price",  F.col("price")+ F.col("charges_converted"))
df= df.fillna(0)

df.printSchema()

df.show()

root
 |-- hotel_name: string (nullable = true)
 |-- location: string (nullable = true)
 |-- distance_from_beirut: string (nullable = true)
 |-- number_reviews: integer (nullable = true)
 |-- distance_from_beach: string (nullable = true)
 |-- room_type: string (nullable = true)
 |-- cancellation_policy: string (nullable = true)
 |-- hotel_policies: string (nullable = true)
 |-- rating: double (nullable = false)
 |-- rating_title: string (nullable = true)
 |-- price: double (nullable = false)
 |-- link: string (nullable = true)
 |-- number_bookings: double (nullable = false)
 |-- sustainability_nb: string (nullable = true)
 |-- summary: string (nullable = false)
 |-- charges_converted: double (nullable = false)
 |-- average_price: double (nullable = false)

+--------------------+--------------------+--------------------+--------------+-------------------+--------------------+-------------------+--------------------+------+------------+------+--------------------+---------------+---------

In [19]:
# around 57% of hotel bookings are made online so these numbers are only restricted to online bookings so I'm going to calculate the number of bookings on average

df= df.withColumn("total_bookings", ((100*F.col("number_bookings"))/57))
df= df.withColumn("total_bookings", F.round("total_bookings"))

df.show(45)

df.write.csv("output.csv", header=True, mode="overwrite")


+--------------------+--------------------+--------------------+--------------+-------------------+--------------------+-------------------+--------------------+------+------------+------+--------------------+---------------+-----------------+--------------------+-----------------+-------------+--------------+
|          hotel_name|            location|distance_from_beirut|number_reviews|distance_from_beach|           room_type|cancellation_policy|      hotel_policies|rating|rating_title| price|                link|number_bookings|sustainability_nb|             summary|charges_converted|average_price|total_bookings|
+--------------------+--------------------+--------------------+--------------+-------------------+--------------------+-------------------+--------------------+------+------------+------+--------------------+---------------+-----------------+--------------------+-----------------+-------------+--------------+
|Radisson Blu Hote...|     Verdun , Beirut|    2 km from centre|

In [21]:
# Machine Learning Analysis

from pyspark.sql.types import StructType, StructField, StringType, DoubleType
from pyspark.sql import SparkSession

spark2 = SparkSession.builder.appName("EmptyDataFrame").getOrCreate()

#  schema of the DataFrame
schema = StructType([
   StructField("col_used", StringType(), True),
   StructField("corr", DoubleType(), True),
])

empty_df = spark2.createDataFrame([], schema)
empty_df.show(10)

a= df.corr("number_bookings","number_reviews")
b= df.corr("number_bookings","total_bookings")
c= df.corr("number_bookings","rating")
d= df.corr("number_bookings","price")
e= df.corr("number_bookings","number_bookings")
f= df.corr("number_bookings","charges_converted")
g= df.corr("number_bookings","average_price")

list= [["number_reviews", a],
       ["total_bookings", b],
       ["rating", c],
       ["price", d],
       ["number_bookings", e],
       ["charges_converted", f],
       ["average_price", g]

]

df_list= spark.createDataFrame(list)

empty_df = empty_df.union(df_list)

from pyspark.sql.functions import desc

empty_df= empty_df.orderBy(desc("corr"))
empty_df.show()



from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression
from pyspark.ml import Pipeline

feature_columns = ["number_reviews", "total_bookings", "rating", "price", "number_bookings", "charges_converted", "average_price"]
assembler = VectorAssembler(inputCols=feature_columns, outputCol="features")
lr = LinearRegression(featuresCol="features", labelCol="expected_number_of_bookings")

pipeline = Pipeline(stages=[assembler, lr])

input_data = spark.read.csv("input_data.csv", header=True, inferSchema=True)
input_data= input_data.drop("hotel_name")
model = pipeline.fit(input_data)


#testing model
predictions = model.transform(df)

predictions.show()



+--------+----+
|col_used|corr|
+--------+----+
+--------+----+

+-----------------+-------------------+
|         col_used|               corr|
+-----------------+-------------------+
|  number_bookings|                1.0|
|   number_reviews| 0.9999999999999997|
|   total_bookings| 0.9999999681304019|
|charges_converted|0.10531164938116573|
|    average_price|0.10473463046277624|
|            price|0.10451223182327249|
|           rating|0.02181625846500293|
+-----------------+-------------------+

+--------------------+--------------------+--------------------+--------------+-------------------+--------------------+-------------------+--------------------+------+------------+------+--------------------+---------------+-----------------+--------------------+-----------------+-------------+--------------+--------------------+------------------+
|          hotel_name|            location|distance_from_beirut|number_reviews|distance_from_beach|           room_type|cancellation_policy|  